This is where LangGraph starts feeling like an actual AI OS — agents that use tools, reason independently, and collaborate through state.

Real-World Use Case:
“AI Research Assistant”
Input: "Compare the weather in Paris and Delhi and tell me if I need an umbrella in either city."

- We’ll design:

1. Planner Agent: decides what to do first (e.g., "Search Paris Weather")
2. Weather Tool Agent: calls SerpAPI or dummy weather function
3. Math Agent: does logic like "if rain > 50%, then need umbrella"
4. LLM Agent: gives final natural language summary



In [24]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType,initialize_agent,load_tools,Tool
from langgraph.graph import StateGraph,END
from pydantic import BaseModel
from typing import Optional,Dict,Any


In [25]:

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=GOOGLE_API_KEY)
# Load serpapi tool for weather queries
tools = load_tools(["serpapi"], llm=llm, serpapi_api_key=SERP_API_KEY)
serp_tool = tools[0]  # single tool from list

Define The State

In [26]:
class WeatherState(BaseModel):
    user_input: str
    plan: Optional[str] = None
    weather_data: Optional[Dict[str,str]] = {}
    logic_results: Optional[Dict[str,str]] = {}
    final_response: Optional[str] = None
    steps: list[str] = []

Nodes Creation

In [27]:
# Planner Agent
def planner_node(state:WeatherState) -> WeatherState:
    state.plan = "Fetch weather for Paris and Delhi"
    state.steps.append("Planner:Decided to get weather for Paris and Delhi")
    return state

# Weather Agent using SerpAPI
def weather_node(state:WeatherState) -> WeatherState:
    cities = ['Paris','Delhi']
    weather_results = {}
    
    for city in cities:
        query = f"Current weather in {city}"
        result = serp_tool.run(query)
        weather_results[city] = result
    
    state.weather_data = weather_results
    state.steps.append("WeatherAgent: Fetched weather using SerpAPI")
    return state

# Math/Decision Agent
import re

def extract_rain_percentage(weather_text: str) -> int:
    match = re.search(r"(\d{1,3})% chance of rain", weather_text)
    return int(match.group(1)) if match else 0

def logic_node(state: WeatherState) -> WeatherState:
    logic = {}
    for city, report in state.weather_data.items():
        rain_percent = extract_rain_percentage(report)
        if rain_percent > 50:
            logic[city] = f"Yes, you should carry an umbrella in {city}."
        else:
            logic[city] = f"No umbrella needed in {city}."
    
    state.logic_results = logic  # Corrected: logic_result -> logic_results
    state.steps.append("MathAgent: Analyzed rain chance for each city.")
    return state

# LLM summary agent
def llm_node(state: WeatherState) -> WeatherState:
    prompt = (
        f"User asked: '{state.user_input}'\n"
        f"Weather Info:\n"
        f"{state.weather_data}\n"
        f"Umbrella Advice:\n"
        f"{state.logic_results}\n"  # Corrected: logic_result -> logic_results
        f"Generate a helpful final response."
    )
    final_answer = llm.invoke(prompt).content
    state.final_response = final_answer
    state.steps.append("LLMAgent: Final summary generated.")
    return state


Graph Building

In [28]:
from langgraph.graph import StateGraph, END

graph = StateGraph(state_schema=WeatherState)

graph.add_node("planner", planner_node)
graph.add_node("weather", weather_node)
graph.add_node("logic", logic_node)
graph.add_node("llm", llm_node)

graph.set_entry_point("planner")
graph.add_edge("planner", "weather")
graph.add_edge("weather", "logic")
graph.add_edge("logic", "llm")
graph.add_edge("llm", END)

app = graph.compile()


In [31]:
initial_state = WeatherState(user_input="Should I carry an umbrella today?")
final_state = app.invoke(initial_state)

print("\n✅ Final Answer:")
print(final_state['final_response'])

print("\n🪜 Agent Steps:")
for step in final_state['steps']:
    print("-", step)


✅ Final Answer:
Okay, based on the weather forecast, here's my advice:

*   **Paris, France:** It's 71°F and cloudy with no precipitation expected. **No umbrella needed in Paris.**
*   **Delhi, India:** It's 93°F and clear with no precipitation expected. **No umbrella needed in Delhi.**

So, wherever you are, you likely don't need an umbrella today.

🪜 Agent Steps:
- Planner:Decided to get weather for Paris and Delhi
- WeatherAgent: Fetched weather using SerpAPI
- MathAgent: Analyzed rain chance for each city.
- LLMAgent: Final summary generated.
